In [1]:
# 基于微软z3 solver求解方程组，使用SageMath进行绘图，Jupyter作为笔记本环境
# 基于节点电压法求解
# 支持调节节点数量，输入电压和接地参考电压
# 除了最后两句其它全是Python代码，只有在安装了z3和Sage的环境下才可正常求解

In [2]:
# 初始化以及配置

from z3 import *
import re

size = 20 # 横/纵节点数量（此程序中最小值为2，最大值取决于电脑性能(•ิ_•ิ)）
Vcc = 10 # 电压（用于z方向上放缩）
GND = 0 # 接地参考电压

In [3]:
#方程求解模块

# 建立节点电压二维数组
u = [[Real(str(i) +","+ str(j)) for j in range(size)] for i in range(size)]

# 新建求解器
s = Solver()

# # **** 电压输入与接地位于同一条边上 ****
# # 设定输入输出电压
# s.add(u[0][0] == Vcc)
# s.add(u[0][size - 1] == GND)

# # 加入剩余两个角落节点的KCL方程
# s.add(2 * u[size-1][size - 1] - u[size-1][size - 2] - u[size-2][size - 1] == 0)
# s.add(2 * u[size - 1][0] - u[size - 2][0] - u[size - 1][1] == 0)
# # **** End ****

# **** 电压输入与接地位于对角线上 ****
# 设定输入输出电压
s.add(u[0][0] == Vcc)
s.add(u[size - 1][size - 1] == GND)

# 加入剩余两个角落节点的KCL方程
s.add(2 * u[0][size - 1] - u[0][size - 2] - u[1][size - 1] == 0)
s.add(2 * u[size - 1][0] - u[size - 2][0] - u[size - 1][1] == 0)
# **** End ****

# 加入边缘节点的KCL方程
for i in range(1, size - 1):
    s.add(3 * u[0][i] - u[1][i] - u[0][i - 1] - u[0][i + 1] == 0)
    s.add(3 * u[size - 1][i] - u[size - 2][i] - u[size - 1][i - 1] - u[size - 1][i + 1] == 0)

    s.add(3 * u[i][0] - u[i][1] - u[i - 1][0] - u[i + 1][0] == 0)
    s.add(3 * u[i][size - 1] - u[i][size - 2] - u[i - 1][size - 1] - u[i + 1][size - 1] == 0)

# 加入内部节点的KCL方程
for i in range(1, size - 1):
    for j in range(1, size - 1):
        s.add(4 * u[i][j] - u[i - 1][j] - u[i + 1][j] - u[i][j - 1] - u[i][j + 1] == 0)
        
# 检查方程组是否成立（一般都是成立的）
print(s.check())

sat


In [4]:
# 绘图模块

# 取得方程组的一组解（此处为唯一解）
res = s.model()
resList = res.decls()

# 生成坐标list
ptList=[]
for item in resList:
    nm = item.name()
    pos = re.split(",", nm)
    val = res.get_interp(item)
    val = eval(val.__str__())
    ptList.append((int(pos[0]),int(pos[1]),val))
#     ptList.append((size-1-int(pos[0]),int(pos[1]),val))

# 绘图
plts = list_plot(ptList,size=50)
plts.show()

Graphics3d Object